# OpenShift Workflow Agent

## Overview

This notebook demonstrates how to build an AI-powered workflow agent that interacts with **OpenShift**, retrieves details about various resources (such as pods or virtual machines), and processes commands using a machine learning model. The agent also integrates with custom tools and persists workflow data using **SQLite**. 

## 1. Setup

### Installing Required Packages

Before we begin, we need to install several Python libraries that are critical for this workflow. These libraries will allow us to:

- **`langchain`**: A framework for building language models and agents.
- **`langgraph`**: A library for creating AI-driven workflows.
- **`requests`**: For sending HTTP requests.
- **`python-dotenv`**: For managing environment variables securely.
- **`kubernetes`**: For interacting with the OpenShift API.

We use the `%pip install` command to ensure all necessary packages are installed.

In [28]:
%pip install langchain langgraph langgraph-checkpoint-sqlite requests termcolor aiohttp python-dotenv kubernetes

Note: you may need to restart the kernel to use updated packages.


## 2. Building Blocks

### OpenShift Configuration

Here, we set up the configuration for connecting to an **OpenShift** cluster. Environment variables are used to securely store connection details, such as the OpenShift API URL, Console URL, and access token. These details are loaded using the `dotenv` library to avoid hardcoding sensitive information in the script.

In [29]:
from dotenv import load_dotenv
import os
from services.openshift_service import OpenShiftService

load_dotenv()

openshift_config = {
    "api_url": os.getenv("OPENSHIFT_API_URL"),
    "console_url": os.getenv("OPENSHIFT_CONSOLE_URL"),
    "token": os.getenv("OPENSHIFT_TOKEN"),
    "inventory_route": os.getenv("OPENSHIFT_INVENTORY_ROUTE"),
}

openshift_service = OpenShiftService(openshift_config=openshift_config)

OpenShift Service connected. API is healthy.


/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-9hg24.yena.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Testing Connectivity to OpenShift

In this section, we test the connection to the OpenShift environment using the credentials provided in the previous configuration step. The goal is to ensure that we can successfully connect to OpenShift, retrieve resources, and interact with the cluster.

We use the following steps:

1. **Connect to OpenShift** using the API URL and access token stored in the environment variables.
2. **Retrieve resources** such as pods or VMs using a custom service class (`OpenShiftService`).
3. **Handle errors** if there's an issue connecting or retrieving data.
4. **Disconnect** from the service once the operation is complete to free up resources.

In [30]:
from typing import Union

def check_openshift_connection() -> Union[bool, str]:
    """
    A tool that checks the connection to the OpenShift Console.

    Returns:
        bool: True if the connection to the OpenShift Console is successful.
        str: Error message if the connection fails.
    """
    try:
        # Create an OpenShiftService instance and attempt to establish a connection
        client = OpenShiftService(openshift_config=openshift_config)

        # Attempt to connect
        connection_successful = (
            openshift_service._test_connection()
        )  # Assuming this method exists
        
        # Return the result
        if connection_successful:
            return True
        else:
            return "Failed to connect to the OpenShift Console."

    except Exception as e:
        return f"Connection error: {str(e)}"

check_openshift_connection()

/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-9hg24.yena.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


OpenShift Service connected. API is healthy.
OpenShift Service connected. API is healthy.


/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-9hg24.yena.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

## 3. Configuring a Simple Model

In this section, we configure the machine learning model that our agent will use to process tasks. The `ModelService` class manages the interaction with the model, allowing the agent to handle tasks such as managing OpenShift resources or retrieving details.

### Model Configuration

We initialize the `ModelService` with a specific model configuration. This step enables our agent to perform model-based tasks using the provided configuration, such as responding to queries or interacting with the OpenShift environment.

In [31]:
from services.model_service import ModelService

# Initialize the service with the model configuration
ollama_service = ModelService(model="llama3.1:8b-instruct-fp16")

## 4. Custom Tools

This section explains how we integrate custom tools into the agent’s workflow. These tools allow the agent to perform specific tasks like managing OpenShift resources. We define tools using the `@tool` decorator from `langchain` that interact with OpenShift and perform actions such as creating or migrating resources.

In [32]:
from typing import List, Dict
from langchain.tools import tool
from services.openshift_service import OpenShiftService

@tool(parse_docstring=True)
def create_migration_plan_tool(
    vm_names: List[str],
    name: str,
    source: str = "vmware",
) -> Union[Dict, str]:
    """
    A tool to create a migration plan for moving virtual machines (VMs) using the forklift.konveyor.io API. This tool only requires VM names, and the corresponding IDs are fetched automatically.

    Args:
        vm_names: A list of VM names to migrate. This arg is mandatory.
        name: The name of the migration plan. This arg is mandatory.
        source: The source provider. Default is 'vmware'.

    Returns:
        dict: The created migration plan if successful.
        str: An error message if the operation fails.
    """
    try:
        # Create an OpenShiftService instance
        OpenShiftService(openshift_config=openshift_config)
        print("\n1/5 Connected to Openshift...\n")

        # Automatically lookup the provider UUID based on the source provider name
        vmware_provider_uuid = openshift_service.lookup_provider_uuid_by_name(
            provider_name="vmware", provider_type="vsphere"
        )
        if isinstance(vmware_provider_uuid, str) and "Error" in vmware_provider_uuid:
            return vmware_provider_uuid  # Return the error if we encountered one

        if vmware_provider_uuid is None:
            return f"Provider '{source}' not found."
        print(f"\n2/5 Now I have the provider {vmware_provider_uuid}...\n")

        # If no VM names are provided, use a default VM
        if vm_names is None:
            return f"Failed to create migration plan: No VM was provided!"

        # Fetch VM IDs for the given VM names
        vms = []
        for vm_name in vm_names:
            vm_id = openshift_service.lookup_vm_id_by_name(
                vmware_provider_uuid, vm_name
            )

            # Assuming error messages are distinct, e.g., returned as dicts or special error codes
            if isinstance(vm_id, dict) and "error" in vm_id:
                return vm_id["error"]  # This handles the error case

            # Check if VM was found
            if vm_id:
                vms.append({"id": vm_id, "name": vm_name})
            else:
                return f"VM '{vm_name}' not found."

        print(f"\n3/5 Now I have all the VM Names and all the VM IDs {vms}...\n")

        network_map, n_uid, network_map_name = openshift_service.create_network_map()
        storage_map, s_uid, storage_map_name = openshift_service.create_storage_map()

        print(f"\n4/5 Now I have the Storage and Network maps {vms}...\n")

        # Call the create_migration_plan method with the gathered information
        result = openshift_service.create_migration_plan(
            network_map_uid=n_uid,
            network_map_name=network_map_name,
            storage_map_uid=s_uid,
            storage_map_name=storage_map_name,
            name=name,
            source=source,
            vms=vms,
        )

        print(f"\n5/5 Migration plan RESPONSE - {result}...\n")
        return result

    except Exception as e:
        return f"Failed to create migration plan: {str(e)}"


@tool(parse_docstring=True)
def start_migration_tool(
    plan_name: str, namespace: str = "openshift-mtv"
) -> Union[Dict, str]:
    """
    A tool to start a migration for a given migration plan by its name.

    Args:
        plan_name: The name of the migration plan. The value should consist of lower case alphanumeric characters, '-' or '.', and must start and end with an alphanumeric character (e.g. 'example.com', regex used for validation is '[a-z0-9]([-a-z0-9]*[a-z0-9])?(\\.[a-z0-9]([-a-z0-9]*[a-z0-9])?)*').
        namespace: The namespace where the migration plan is located. Default is 'openshift-mtv'.


    Returns:
        dict: The migration start response if successful.
        str: An error message if the operation fails.
    """
    try:
        # Create an OpenShiftService instance
        OpenShiftService(openshift_config=openshift_config)
        print("\n1/4 Connected to Openshift...\n")

        # Retrieve the full migration plan object by its name
        migration_plan = openshift_service.get_migration_plan_by_name(
            plan_name=plan_name, namespace=namespace
        )

        if isinstance(migration_plan, str) and "Error" in migration_plan:
            return migration_plan  # Return the error if we encountered one

        if migration_plan is None:
            return f"Migration plan '{plan_name}' not found."

        # Retrieve the UID of the migration plan
        plan_uid = migration_plan["metadata"]["uid"]
        print(f"\n2/4 Retrieved migration plan UID: {plan_uid}...\n")

        # Check the status conditions to verify if the plan is ready
        ready_status = False
        conditions = migration_plan.get("status", {}).get("conditions", [])
        for condition in conditions:
            if condition.get("type") == "Ready" and condition.get("status") == "True":
                ready_status = True
                break

        if not ready_status:
            return f"Migration plan '{plan_name}' is not ready."

        print(f"\n3/4 Migration plan is ready. Starting migration...\n")

        # Start the migration using the retrieved plan UID
        result = openshift_service.start_migration(
            plan_name=plan_name, plan_uid=plan_uid, namespace=namespace
        )

        print(f"\n4/4 Migration started: {result}...\n")
        return result

    except Exception as e:
        return f"Failed to start migration: {str(e)}"

In [33]:
import json
from utils.general.tools import (
    get_tools_name,
    get_tools_description_json,
)

tools = [create_migration_plan_tool, start_migration_tool]

tools_name = get_tools_name(tools)
tools_description_json = get_tools_description_json(tools)

tools_description = json.dumps(tools_description_json, indent=4)

print(tools_description)

[
    {
        "name": "create_migration_plan_tool",
        "description": "A tool to create a migration plan for moving virtual machines (VMs) using the forklift.konveyor.io API. This tool only requires VM names, and the corresponding IDs are fetched automatically.",
        "parameters": {
            "vm_names": {
                "param_type": "str",
                "description": "A list of VM names to migrate. This arg is mandatory.",
                "required": true
            },
            "name": {
                "param_type": "str",
                "description": "The name of the migration plan. This arg is mandatory.",
                "required": true
            },
            "source": {
                "param_type": "str",
                "description": "The source provider. Default is 'vmware'.",
                "required": false
            }
        }
    },
    {
        "name": "start_migration_tool",
        "description": "A tool to start a migration for a give

## 5. Workflow Graph Setup

### Node: `react_node_function`

This function handles the execution of tasks by the ReAct agent. It takes the current state of the agent and processes the user request using the available tools (such as listing resources or retrieving details). The response from the ReAct agent is returned as the output.

In [34]:
from state.agent_graph import AgentGraphState
from agent.react_agent import ReactAgent

def react_node_function(state: AgentGraphState):
    react_agent = ReactAgent(
        state=state,
        role="REACT_AGENT",
        tools=tools,
        ollama_service=ollama_service,
    )

    return react_agent.react(user_request=state["input"])

### State Graph

In this step, we define a `StateGraph` using the `AgentGraphState`. The graph determines the flow of tasks through various agent nodes, using an AI-driven ReAct agent to process tasks based on the inputs it receives.

In [35]:
from langgraph.graph import StateGraph, START, END


def create_graph() -> StateGraph:
    """
    Create the state graph by defining nodes and edges.

    Returns:
    - StateGraph: The compiled state graph ready for execution.
    """
    graph = StateGraph(AgentGraphState)

    # Add nodes
    graph.add_node("react_agent", react_node_function)

    # Define the flow of the graph
    graph.add_edge(START, "react_agent")
    graph.add_edge("react_agent", END)

    return graph

### Initializing Sqlite Persistence for Graph State

In this step, we set up persistence for the agent's state using SQLite. This allows us to store the state in an SQLite database, ensuring that the workflow’s context is preserved even after multiple interactions. The `SqliteSaver` class handles the saving and retrieval of this state.

In [36]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3


def from_conn_stringx(
    cls,
    conn_string: str,
) -> "SqliteSaver":
    return SqliteSaver(conn=sqlite3.connect(conn_string, check_same_thread=False))


SqliteSaver.from_conn_stringx = classmethod(from_conn_stringx)

memory = SqliteSaver.from_conn_stringx(":memory:")

## 6. Executing the Workflow

Here, we execute the workflow by compiling the state graph and defining the inputs for the agent. The agent processes user requests (such as listing resources) and streams the events as it progresses through the workflow. This allows us to see the state of the agent at each step.

### Query 1: Create a Migration Plan for a VM

In this query, we instruct the AI agent to perform a single task: **create a migration plan** for a virtual machine (VM) named `database`. The agent is explicitly told to focus only on creating the migration plan and not to retrieve any additional details about the VM.

In [37]:
# Create the graph and compile the workflow
graph = create_graph()
workflow = graph.compile(checkpointer=memory)
print("Graph and workflow created.")

# Define workflow parameters
iterations = 10
verbose = True
config = {"configurable": {"thread_id": "1"}}

vm_name = "database"
query = f"You should execute only one task. THIS IS A SINGLE TASK: Use the create_migration_plan_tool to create a migration plan for the vm named '{vm_name}'! REMEMBER: DON'T DO ANY ADDITIONAL STEP! DO NOT START OR EXECUTE THE MIGRATION YET! "
dict_inputs = {"input": query}
limit = {"recursion_limit": iterations}

# Execute the workflow and print state changes
for event in workflow.stream(dict_inputs, config):
    if verbose:
            print("\nEvent:", event)
    else:
        print("\n")

Graph and workflow created.
<|start_header_id|>user<|end_header_id|>

You should execute only one task. THIS IS A SINGLE TASK: Use the create_migration_plan_tool to create a migration plan for the vm named 'database'! REMEMBER: DON'T DO ANY ADDITIONAL STEP! DO NOT START OR EXECUTE THE MIGRATION YET! <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "To create a migration plan, I need to use the create_migration_plan_tool with the correct input.",
    "action": "create_migration_plan_tool",
    "action_input": {
        "vm_names": [
            "database"
        ],
        "name": "migration-plan-database",
        "source": "vmware"
    }
}<|eot_id|>
<|python_tag|>create_migration_plan_tool.call({'vm_names': ['database'], 'name': 'migration-plan-database', 'source': 'vmware'})
<|eom_id|>


/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-9hg24.yena.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


OpenShift Service connected. API is healthy.

1/5 Connected to Openshift...



/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forklift-inventory-openshift-mtv.apps.rosa-9hg24.yena.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



2/5 Now I have the provider a1699c04-47c7-4864-852e-f3877bc889e9...



...


3/5 Now I have all the VM Names and all the VM IDs [{'id': 'vm-88352', 'name': 'database'}]...



/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-9hg24.yena.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-9hg24.yena.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



4/5 Now I have the Storage and Network maps [{'id': 'vm-88352', 'name': 'database'}]...



/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forklift-inventory-openshift-mtv.apps.rosa-9hg24.yena.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forklift-inventory-openshift-mtv.apps.rosa-9hg24.yena.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ap


5/5 Migration plan RESPONSE - {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'Plan', 'metadata': {'annotations': {'populatorLabels': 'True'}, 'creationTimestamp': '2024-09-06T14:44:34Z', 'generation': 1, 'managedFields': [{'apiVersion': 'forklift.konveyor.io/v1beta1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:spec': {'.': {}, 'f:map': {'.': {}, 'f:network': {}, 'f:storage': {}}, 'f:provider': {'.': {}, 'f:destination': {}, 'f:source': {}}, 'f:targetNamespace': {}, 'f:vms': {}}}, 'manager': 'OpenAPI-Generator', 'operation': 'Update', 'time': '2024-09-06T14:44:34Z'}], 'name': 'migration-plan-database', 'namespace': 'openshift-mtv', 'resourceVersion': '2008751', 'uid': '37a38c7f-a87e-4e88-83c0-0a6780dd8093'}, 'spec': {'map': {'network': {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'NetworkMap', 'name': 'vmware-k4bhm', 'namespace': 'openshift-mtv', 'uid': '0e5f7d08-6cd8-4369-85fd-a2b9309f115a'}, 'storage': {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'StorageMa

### Saving the Agent's Output

After the agent completes its task (e.g., listing VMs), we save the output to a JSON file for future reference or analysis. We use the `save_json_to_file` function to handle this, ensuring that the output is saved in a well-formatted, pretty-printed JSON file.

**Function**: `remove_prefix`
This helper function extracts a removes the `I have the answer: ` prefix.

In [38]:
from utils.general.helpers import save_json_to_file, remove_prefix

# Assuming workflow.get_state() returns some structured data
state_snapshot = workflow.get_state(config)

# Check if there's any response
if state_snapshot.values["tool_response"]:

    # Extract the list of VMs
    extracted_json_string = remove_prefix(state_snapshot.values["tool_response"][-1].content)
    cleaned_content = extracted_json_string.strip().rstrip(
        "."
    )  # Removing trailing period

    cleaned_content = cleaned_content.replace("'", '"')
    extracted_json = json.loads(cleaned_content)

    print(json.dumps(extracted_json, indent=4))

    save_json_to_file(extracted_json, "data/migration_plan_output.json")
else:
    print("There's no migration plan to save!")

migration_plan_name = extracted_json["metadata"]["name"]

{
    "apiVersion": "forklift.konveyor.io/v1beta1",
    "kind": "Plan",
    "metadata": {
        "annotations": {
            "populatorLabels": "True"
        },
        "creationTimestamp": "2024-09-06T14:44:34Z",
        "generation": 1,
        "managedFields": [
            {
                "apiVersion": "forklift.konveyor.io/v1beta1",
                "fieldsType": "FieldsV1",
                "fieldsV1": {
                    "f:spec": {
                        ".": {},
                        "f:map": {
                            ".": {},
                            "f:network": {},
                            "f:storage": {}
                        },
                        "f:provider": {
                            ".": {},
                            "f:destination": {},
                            "f:source": {}
                        },
                        "f:targetNamespace": {},
                        "f:vms": {}
                    }
                },
        

## 8. Start the Migration Plan

### Query 2: Start the Migration Plan

Here, we provide a query asking the agent to start a migration plan for specific OpenShift resources. The agent processes the request, and the migration is triggered within the OpenShift environment.

In [39]:
query = f"You should execute only one task. THIS IS A SINGLE TASK: Use the start_migration_plan_tool to start the {migration_plan_name}! REMEMBER: DON'T DO ANY ADDITIONAL STEP! DO NOT CREATE THE MIGRATION! "
dict_inputs = {"input": query}
limit = {"recursion_limit": iterations}

# Execute the workflow and print state changes
for event in workflow.stream(dict_inputs, config):
    if verbose:
        print("\nEvent:", event)
    else:
        print("\n")

<|start_header_id|>user<|end_header_id|>

You should execute only one task. THIS IS A SINGLE TASK: Use the start_migration_plan_tool to start the migration-plan-database! REMEMBER: DON'T DO ANY ADDITIONAL STEP! DO NOT CREATE THE MIGRATION! <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "To start the migration plan 'migration-plan-database', I need to use the start_migration_tool.",
    "action": "start_migration_plan_tool",
    "action_input": {
        "plan_name": "migration-plan-database"
    }
}<|eot_id|>
<|python_tag|>start_migration_plan_tool.call({'plan_name': 'migration-plan-database'})
<|eom_id|>
too many values to unpack (expected 2)
<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "To start the migration plan 'migration-plan-database', I need to use the start_migration_tool.",
    "action": "start_migration_tool",
    "action_input": {
        "plan_name": "migration-plan-database"
    }
}<|eot_id|>
<|python_tag|>start_migration_too

/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-9hg24.yena.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


OpenShift Service connected. API is healthy.

1/4 Connected to Openshift...


2/4 Retrieved migration plan UID: 37a38c7f-a87e-4e88-83c0-0a6780dd8093...


3/4 Migration plan is ready. Starting migration...


4/4 Migration started: {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'Migration', 'metadata': {'creationTimestamp': '2024-09-06T14:44:50Z', 'generateName': 'migration-plan-database-', 'generation': 1, 'managedFields': [{'apiVersion': 'forklift.konveyor.io/v1beta1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:ownerReferences': {'.': {}, 'k:{"uid":"37a38c7f-a87e-4e88-83c0-0a6780dd8093"}': {}}}, 'f:spec': {'.': {}, 'f:plan': {}}}, 'manager': 'python-requests', 'operation': 'Update', 'time': '2024-09-06T14:44:50Z'}], 'name': 'migration-plan-database-h2tcz', 'namespace': 'openshift-mtv', 'ownerReferences': [{'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'Plan', 'name': 'migration-plan-database', 'uid': '37a38c7f-a87e-4e88-83c0-0a6780dd8

## Conclusion

This notebook demonstrates how to build an AI-powered agent that interacts with an OpenShift environment, retrieves resource information, and processes tasks using a machine learning model. The agent can perform custom tasks based on defined tools and persist its state using SQLite, making it adaptable for complex workflows in real-world environments.